In [214]:
from natasha import NewsNERTagger
from natasha import MorphVocab, NewsEmbedding, NewsMorphTagger
from natasha import Doc, Segmenter
import json
import pandas as pd
import re
import pymorphy2
analyzer = pymorphy2.MorphAnalyzer()
embedding = NewsEmbedding()
segmenter = Segmenter()
morph_tagger = NewsMorphTagger(embedding)
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(embedding)

In [215]:
df = pd.read_csv('news_new.csv', encoding='utf-8')

Возьмём датасет содержащий статьи и их названия

In [216]:
df.sample(6)

,Unnamed: 0,title,text
16496,16496,"Миссия из Италии, Германии и Греции пролетит н...","МОСКВА, 9 мар — РИА Новости. Совместная мисси..."
12406,12406,Почти две тысячи подмосковных медиков заразили...,"МОСКВА, 18 мая — РИА Новости. Почти две тысяч..."
7868,7868,В Тбилиси задержали сына экс-премьера Грузии,"ТБИЛИСИ, 2 авг - РИА Новости. Министерство вн..."
19809,19809,"Исследование показало, что четверть автомобили...","МОСКВА, 13 янв - РИА Новости/Прайм. Почти 20%..."
20276,20276,"Тэрон Эджертон завоевал ""Золотой глобус"" за лу...","ВАШИНГТОН, 6 янв – РИА Новости. Актер Тэрон Э..."
8151,8151,Украина пожаловалась генсеку ООН на морской па...,"МОСКВА, 29 июл — РИА Новости. Украина обрат..."


In [217]:
def get_ner(transcript):
    script = Doc(re.sub(r'\((.*?)\)', "", transcript))
    script.segment(segmenter)
    script.tag_morph(morph_tagger)
    for token in script.tokens:
        token.lemmatize(morph_vocab)
    script.tag_ner(ner_tagger)
    for span in script.spans:
        span.normalize(morph_vocab)
    named_ents = [(i.text, i.type, i.normal) for i in script.spans]
    normed_ents = []
    for word, tag, norm in named_ents:
        if len(word.split()) == 1 and tag == "LOC":
            for gram in range(len(analyzer.parse(word))):
                if "Geox" in analyzer.parse(word)[gram].tag:
                    normed_ents.append((analyzer.parse(word)[gram].normal_form))
                    break
                elif gram == len(analyzer.parse(word)) - 1:
                    normed_ents.append((norm.lower().strip(".,!?;-")))
        else:
            normed_ents.append((norm.lower().strip(".,!?;-")))
    return sorted(normed_ents)

In [220]:
df["named_entities"] = df.apply(lambda df: get_ner(df["text"]), axis=1)

In [222]:
df.sample(6)

,Unnamed: 0,title,text,named_entities
5234,5234,Собянин пообещал к концу недели развернуть сис...,Власти Москвы планируют к концу недели полност...,"[интерфакс, москва, москва, москва, москва, мо..."
14608,14608,НАПКА не прогнозирует массовых дефолтов из-за ...,"МОСКВА, 11 апр - РИА Новости. Число должников...","[мехтиев, москва, напка, национальная ассоциац..."
10175,10175,В США призвали СБ ООН продлить оружейное эмбар...,"ООН, 25 июн - РИА Новости. Спецпредставитель ...","[антониу гутерреш, брайан хук, евросоюз, иран,..."
2154,2154,Подсчитаны потери России от коронавируса,Еврокомиссия в собственном докладе подсчита...,"[еврокомиссия, еврокомиссия, россия, россия, р..."
19147,19147,Трамп обсудил с Джонсоном сотрудничество в ряд...,"ВАШИНГТОН, 25 янв - РИА Новости. Президент СШ...","[белый дом, борис джонсон, борис джонсон, ваши..."
20148,20148,"СМИ: совещание у Трампа завершилось, министры ...","РИА НОВОСТИ Совещание у Трампа завершилось, ми...","[cnn, белый дом, риа новости, трамп]"


Посмотрим, сколько всего именованных сущностей в корпусе.

In [224]:
ner_voc = []
for row in df.named_entities.tolist():
    ner_voc.extend(row)
len(ner_voc), len(set(ner_voc))

(365336, 55811)

In [225]:
vocabulary = sorted(set(ner_voc))
corpus = ner_df.ner.apply(str).tolist()

In [226]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans

In [241]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
X = pipe.fit_transform(corpus)
km = KMeans(n_clusters=30, init='k-means++', max_iter=600, 
            algorithm="full", precompute_distances=True)

In [242]:
km.fit(X)

c:\users\redic\appdata\local\programs\python\python38\lib\site-packages\sklearn\cluster\_kmeans.py:786: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25). It has no effect
  warnings.warn("'precompute_distances' was deprecated in version "


KMeans(algorithm='full', max_iter=600, n_clusters=30, precompute_distances=True)

In [243]:
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.6286701937984022
1.493350935978347


С увеличение количества кластеров силуэт больше стремится к 1 (а значит разбиение становится лучше)

In [244]:
ner_df["label"] = km.predict(X)

In [245]:
ner_df["label"].value_counts()

0     9280
4     5771
6      528
1      425
15     330
20     255
12     252
9      214
7      156
2      147
24     138
11     132
17     131
21     127
8      117
18     117
29      96
16      92
14      92
3       90
23      84
25      79
19      73
5       56
13      53
27      44
28      37
22      29
10      28
26      24
Name: label, dtype: int64

Так как в нулевом и четвертом кластере слишком много данных, то это может означать, что он собрал объекты, которые сложно куда-то определить.

In [251]:
ner_df.query("label == 5").sample(8)

,text_num,ner,label
14242,15956,"[лондон, наталья копылова, риа новости]",5
12446,14135,"[лондон, наталья копылова, риа новости]",5
9517,11169,"[лондон, наталья копылова, риа новости]",5
17780,19564,"[наталья дембинск, риа новости]",5
10687,12351,"[лондон, наталья копылова, риа новости]",5
6375,7979,"[наталья дембинска, риа новости]",5
16819,18587,"[лондон, наталья копылова, риа новости]",5
18694,20493,"[наталья дембинска, риа новости]",5


In [252]:
ner_df.query("label == 8").sample(8)

,text_num,ner,label
18612,20408,"[20408 каир, риа новости]",8
13109,14809,"[14809 каир, риа новости]",8
14602,16321,"[16321 каир, риа новости, сша]",8
17118,18889,"[18889 каир, риа новости]",8
17214,18987,"[18987 каир, риа новости]",8
15368,17102,"[17102 каир, риа новости]",8
13332,15035,"[15035 каир, риа новости]",8
7788,9413,"[9413 каир, риа новости]",8


In [253]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = pipe[0].get_feature_names()
for i in range(30):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: риа новости „агора джейк джейми джейкоб элорди джейкоб фрей джейкоб блейку джейкоб блейка джейкоб блейк
Cluster 1: сша новости риа сенат россия иран росс китай япония газа
Cluster 2: киев новости риа украина симферополь служба снбо львов польша совет
Cluster 3: суд москва верховный басманный пресненский конституционный новости риа таганский россия
Cluster 4: новости риа сергей берлин алексей михаил китай белград брюссель красноярск
Cluster 5: наталья лондон новости риа госдума поклонская непряева штурм водянова дума
Cluster 6: россия новости риа дмитрий министерство банк мид минфин цик мвд
Cluster 7: оон новости риа россия совбез сша иран великобритания китай мид
Cluster 8: каир новости риа египет алжир тунис ливийская мид эфиопия марга
Cluster 9: пекин новости риа китай анна кнр ухань сша гонконг росси
Cluster 10: youtube новости риа канал „агора джейк джейкоб элорди джейкоб фрей джейкоб блейку джейкоб блейка
Cluster 11: владимир путин россия рф турция новости риа госдума м

Наиболее частотные слова в кластере выглядят адекватно. По некоторым кластерам можно даже определить про какие события писали в новостных статьях. В целом тематика и направленность датасета (политика, международные отношения) здесь тоже показаны.